In [1]:
import tensorflow as tf
import numpy as np
import json
import re
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, GRU, LSTM
from keras.preprocessing.text import tokenizer_from_json

In [2]:
# import preprocessed data

with open('./preprocessed_data/questions.json', 'r') as f:
    json_data = json.load(f)
    question_corpus = tokenizer_from_json(json_data)
    f.close()

with open('./preprocessed_data/answers.json', 'r') as f:
    json_data = json.load(f)
    answer_corpus = tokenizer_from_json(json_data)
    f.close()

npzfile = np.load('./preprocessed_data/data.npz') 

In [3]:
# some unknown reason make the corpus contain all words and labels in raw data rather than a vocabulary with limited size
# so we have to build the dict manually
q_word2ind={e:i for e,i in question_corpus.word_index.items() if i <= 8000}
q_ind2word={e:i for i,e in q_word2ind.items()}
a_word2ind={e:i for e,i in answer_corpus.word_index.items() if i <= 8000}
a_ind2word={e:i for i,e in a_word2ind.items()}

In [ ]:
# define encoder
# notice that the encoder at here is totally same with the encoder in training model
def create_encoder(inputdim, embeddingsize, inputlen, n_units):

    encoder_input = Input((inputlen,))
    encoder_embed = Embedding(inputdim+1, embeddingsize)(encoder_input)
    encoder = LSTM(n_units, return_state = True)
    _, encoder_h, encoder_c = encoder(encoder_embed)
    
    encoder=Model(encoder_input, [encoder_h,encoder_c])
    
    return encoder

In [4]:
# define decoder
# notice that we use inference model at here, which is a little different with the decoder in training model
def create_decoder(inputdim, embeddingsize,inputlen, n_units):
    # the size of input at here is 1 because we want to predict the answer step by step, each time only input 1 word
    decoder_input = Input((1,))
    initial_stateh = Input((n_units,))
    initial_statec = Input((n_units,))
    encoder_state = [initial_stateh,initial_statec]
    decoder_embed = Embedding(inputdim+1, embeddingsize,input_length = 1)(decoder_input)
    decoder = LSTM(n_units, return_sequences = True, return_state = True)
    # in training model, we dont use the state h & c. but in inference model, we do
    decoder_output, decoder_h, decoder_c = decoder(decoder_embed,initial_state = encoder_state)
    decoder_dense = Dense(inputdim+1, activation = 'softmax')
    decoder_output_ = decoder_dense(decoder_output)
    
    decoder=Model([decoder_input,initial_stateh,initial_statec],decoder_output_)
    
    return decoder

In [5]:
# define hyperparameters

N_Unit = 256
EmbeddingSize = 128
VocabSize = 8000 
QuestionLen = npzfile['arr_0'].shape[1]
AnswerLen = npzfile['arr_1'].shape[1]

In [6]:
# define hyperparameters

BatchSize = 64
N_Unit = 256
EmbeddingSize = 128
VocabSize = 8000 
# theoretically, vocabulary size should be len(question_corpus.word_index)+1. 
# however, seems like the 'num_words' didnt filter the tokenizer. so we assign the number manually
QuestionLen = npzfile['arr_0'].shape[1]
AnswerLen = npzfile['arr_1'].shape[1]

In [7]:
encoder=create_encoder(VocabSize,EmbeddingSize,QuestionLen,N_Unit)

In [8]:
encoder.load_weights('./trained_model/lstm_encoder_weights.h5')

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 6)]               0         
_________________________________________________________________
embedding (Embedding)        (None, 6, 128)            1024128   
_________________________________________________________________
gru (GRU)                    [(None, 256), (None, 256) 296448    
Total params: 1,320,576
Trainable params: 1,320,576
Non-trainable params: 0
_________________________________________________________________


In [9]:
decoder=create_decoder(VocabSize,EmbeddingSize,N_Unit)

In [10]:
decoder.load_weights('./trained_model/lstm_decoder_weights.h5')

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 128)       1024128     input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256)]        0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, 1, 256), (No 296448      embedding_1[0][0]                
                                                                 input_3[0][0]              

In [15]:
def clean_text(text):

    # remove unnecessary characters in sentences
    
    text = text.lower().strip()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"there's", "there is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r'[" "]+', " ", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

In [16]:
def evaluate(sentence):
    
    sentence=clean_text(sentence) # clean the input text
    encoder_inputs=[]
    # convert the input text to index sequence and use unk replace the word not in vocabulary
    for word in sentence.split():
        if word in q_word2ind.items():
            encoder_inputs.append(q_word2ind[word])
        elif word not in q_word2ind.items():
            encoder_inputs.append(q_word2ind['unk'])

    encoder_inputs=tf.keras.preprocessing.sequence.pad_sequences([encoder_inputs],maxlen=QuestionLen,padding='post')
    encoder_inputs = tf.convert_to_tensor(encoder_inputs)
    encoder_h,encoder_c=encoder(encoder_inputs)
    
    # initialize the decoder input
    decoder_inputs=tf.expand_dims([a_word2ind['bos']], 0)
    hidden_h,hidden_c=encoder_h,encoder_c
    
    result=''
    for t in range(AnswerLen):
        pred,state_h,state_c=decoder([decoder_inputs,hidden_h,hidden_c])
        pred=np.squeeze(pred)
        pred_ind=tf.math.argmax(pred).numpy()
 
        if pred_ind==0:   # in dict we dont have label 0, so if the predict is padding stuff, stop the loop
            return result
        elif a_ind2word[pred_ind]=='eos': # once we get the eos symbol, stop the loop
            return result
        result+=a_ind2word[pred_ind] + ' '
        decoder_inputs=tf.expand_dims([pred_ind],0) # pass the predict index and state vectors to the next input       
        hidden_h,hidden_c=state_h,state_C
    return result

In [17]:
while True:
    inputs = input('User :> ')
    if inputs == 'quit':
        break

    result = evaluate(inputs)

    print('Bot :> ' + result)